In [287]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import ast
from typing import Dict, List
from catboost import CatBoostClassifier
import json
from pandas import json_normalize
from tqdm import tqdm_notebook as tqdm
import catboost

In [288]:
pd.set_option("display.max_columns", None)
pd.options.mode.chained_assignment = None

In [289]:
events = pd.read_csv('../input/hackaton-ufc/data/0.events_raw.csv')
fighters = pd.read_csv('../input/hackaton-ufc/data/0.fighters_raw.csv')
fighters_cols = [
    "id",
    "name",
    "weight",
    "height",
    "armSpan",
    "legSwing",
    "weightCategory.id",
    "weight_category_name",
    "dateOfBirth",
    "country",
    "city",
    "timezone",
]

In [290]:
## Исследование данных о бойцах

In [291]:
fighters.head()

In [292]:
fighters.info()

In [293]:
fighters.columns

In [294]:
fighters['id'].count()

In [295]:
fighters.duplicated().sum()

In [296]:
fighters['dateOfBirth'] = pd.to_datetime(fighters["dateOfBirth"])

In [297]:
fighters['country'].unique()

In [298]:
len(fighters['country'].unique())

In [299]:
usa_state_names = [
    "Alaska",
    "Alabama",
    "Arkansas",
    "American Samoa",
    "Arizona",
    "California",
    "Colorado",
    "Connecticut",
    "District ",
    "of Columbia",
    "Delaware",
    "Florida",
    "Floirda"
    "Georgia",
    "Guam",
    "Hawaii",
    "Iowa",
    "Idaho",
    "Illinois",
    "Indiana",
    "Kansas",
    "Kentucky",
    "Louisiana",
    "Saint Louis",
    "Los Angeles"
    "Massachusetts",
    "Maryland",
    "Maine",
    "Michigan",
    "Minnesota",
    "Missouri",
    "Mississippi",
    "Montana",
    "North Carolina",
    "North Dakota",
    "Nebraska",
    "New Hampshire",
    "New Jersey",
    "New Mexico",
    "Nevada",
    "New York",
    "Ohio",
    "Oklahoma",
    "Oregon",
    "Pennsylvania",
    "Puerto Rico",
    "Rhode Island",
    "South Carolina",
    "South Dakota",
    "Tennessee",
    "Texas",
    "Utah",
    "Virginia",
    "Virgin Islands",
    "Vermont",
    "Washington",
    "Wisconsin",
    "West Virginia",
    "Wyoming",
    "Massachusetts"
    
    
]


Штаты и United States  заменить на USA, Дагестан - это Россия, Wales - Великобритания, есть Советский Союз, есть England и United Kingdom, Saint Pierre and Miquelon - Франция, 'Los Angeles', 'Saint Louis' - USA, Ontario - Канада, Ukrainian SSR и Ukraine, Floirda - США, Rostov-On-Don - Россия, Califorina- USA, Democratic Republic of Congo/Democratic Republic Of The Congo/Republic of Congo, Bosnia/Bosnia Herzegovina, Armenian SSR/Armenia, Colombia/ British Columbia, Quebec - Канада


In [300]:
fighters.loc[fighters['country'] == 'United States', 'country'] = 'USA'
fighters.loc[fighters['country'].isin(usa_state_names), 'country'] = 'USA'


len(fighters['country'].unique())

In [301]:
fighters.loc[fighters['country'] == 'Dagestan', 'country'] = 'Russia'
fighters.loc[fighters['country'] == 'Rostov-On-Don', 'country'] = 'Russia'
fighters.loc[fighters['country'] == 'Wales', 'country'] = 'United Kingdom'
fighters.loc[fighters['country'] == 'England', 'country'] = 'United Kingdom'
fighters.loc[fighters['country'] == 'Saint Pierre and Miquelon', 'country'] = 'France'
fighters.loc[fighters['country'] == 'Ontario', 'country'] = 'Canada'
fighters.loc[fighters['country'] == 'Quebec', 'country'] = 'Canada'
fighters.loc[fighters['country'] == 'Ukrainian SSR', 'country'] = 'Soviet Union'
fighters.loc[fighters['country'] == 'Democratic Republic Of The Congo', 'country'] = 'Democratic Republic of Congo'
fighters.loc[fighters['country'] == 'Republic of Congo', 'country'] = 'Democratic Republic of Congo'
fighters.loc[fighters['country'] == 'Bosnia Herzegovina', 'country'] = 'Bosnia'
fighters.loc[fighters['country'] == 'Armenian SSR', 'country'] = 'Soviet Union'
fighters.loc[fighters['country'] == 'British Columbia', 'country'] = 'Colombia'
fighters.loc[fighters['country'] == 'Holland', 'country'] = 'Netherlands'


In [302]:
len(fighters['country'].unique())

In [303]:
fighters['country'].unique()

In [304]:
len(fighters['country'].unique())

In [305]:
fighters.describe()


In [306]:
fighters['armSpan'].describe()

In [307]:
fighters['armSpan'].plot.box()

In [308]:
fighters[fighters['armSpan'] > 210]['armSpan'].count()

In [309]:
fighters[fighters['armSpan'] > 210]['armSpan']

In [310]:
#fighters['avgFightTime'] = pd.to_datetime(fighters['avgFightTime'], format='%M:%S').dt.time

In [311]:
fighters['avgFightTime'].describe()

In [312]:
fighters['legSwing'].describe()

In [313]:
fighters['legSwing'].plot.box()

In [314]:
fighters[fighters['legSwing'] < 85]['legSwing']

In [315]:
fighters[fighters['legSwing'] > 120]['legSwing']

In [316]:
fighters['height'].describe()

In [317]:
fighters['height'].plot.box()

In [318]:
fighters[fighters['height'] > 220]['height']

In [319]:
fighters[fighters['height'] < 154]['height']

In [320]:
fighters['weight'].describe()

In [321]:
fighters['weight'].plot.box()

In [322]:
fighters[fighters['weight'] < 52]['weight']

In [323]:
fighters[fighters['weight'] > 200]['weight']

In [324]:
fighters['weight'].isnull().sum() 

In [325]:
fighters['height'].isnull().sum()

In [326]:
fighters['legSwing'].isnull().sum()

In [327]:
fighters['armSpan'].isnull().sum()

In [328]:
fighters['id'].count()

### Перед удалением выбросов обработать пропуски

### Удалим выбросы в height, weight, legSwing, armSpan    

In [329]:
#fighters = fighters[(fighters['legSwing'] < 122) & (fighters['legSwing'] > 80) & (fighters['armSpan'] < 220) & (fighters['height'] < 227) & (fighters['weight'] < 200)]

In [330]:
fighters['id'].count()

In [331]:
fighters.describe()

https://ufc.ru/intro-to-mma

armSpan Arm span or reach (sometimes referred to as wingspan, or spelled "armspan") is the physical measurement of the length from one end of an individual's arms (measured at the fingertips) to the other when raised parallel to the ground at shoulder height at a 90° angle.

legSwing размах ноги

Draw ничья

significantStrikesMissedPerMinute Значительные удары Significant Strikes are basically any strike that significantly improves your chance of winning the fight. That basically boils down to any strike thrown to the legs, body, or head with enough force to be considered damaging.

Judges' Decision (U-Dec, S-Dec):
When a fight is not finished by a submission, KO, or TKO, the judges deliver their scorecards for a final decision. Когда бой не завершается сабмишеном, нокаутом или техническим нокаутом, судьи выносят окончательное решение на табло.

Нокаут ( англ. КО ) : боец оказывается в бессознательном состоянии в результате одного или нескольких точных ударов из положения стоя, после которого (ых) бой немедленно остановлен рефери, без возможности добивания соперника на земле.



Технический нокаут (англ. TKO): остановка боя третьим лицом в результате потери одним из бойцов способности продолжать бой. Технические нокауты могут подразделяться на три категории:

Остановка рефери (англ. Referee stoppage). Рефери решает, что один из бойцов не может осмысленно защищаться, и останавливает бой.

Остановка врачом (врач, присутствующий возле ринга, решает, что дальнейшее участие одного из бойцов ставит жизнь или здоровье этого участника под угрозу. Например, травмы или обильное кровотечение)

Остановка «углом» (англ. Corner stoppage). Угловой секундант бойца сигнализирует об остановке боя.

DQ - дисквалификация. Использование запрещённого приема в поединке.
Болевые и удушающие относятся к ТКО, то есть это добровольная сдача бойца или остановка боя арбитром.

Submission (SUB): 
Physical or verbal tap out by one of dozens of methods of submission. 

In [332]:
fighters['disciplines'].unique()

In [333]:
fighters['weightCategory.name'].unique()

Наилегчайший вес — от 53 до 57 килограммов.
Легчайший вес — от 57 до 61кг
Полулёгкий вес — от 61 до 66кг
Лёгкий вес — от 66 до 70кг
Полусредний вес — от 70 до 77кг
Средний вес — от 77 до 84 кг
Полутяжёлый вес — от 84 до 94 кг
Тяжёлый вес — от 93 до 120кг

Женский минимальный вес (Strawweight) — до 115 фунтов (52 кг);
Наилегчайший вес (Flyweight) — до 125 фунтов (57 кг);
Легчайший вес (Bantamweight) — до 135 фунтов (61 кг);
Полулёгкий вес (Bantamweight) — до 145 фунтов (66 кг).

In [334]:
# Посмотрим имена категорий веса по id
weight_id_name = fighters.loc[:,['weightCategory.id', 'weightCategory.name']]
weight_id_name.drop_duplicates().sort_values(by=['weightCategory.id'])

In [335]:
fighters.groupby(by='weightCategory.id').mean()['weight']

In [336]:
# Количество бойцов в каждой весовой категории
fighters['weightCategory.name'].value_counts()

In [337]:
plt.figure(figsize=(10,5))
bar = sns.countplot(fighters['weightCategory.name'])
plt.xticks(rotation=90)
ax = plt.gca()
y_max = fighters['weightCategory.name'].value_counts().max() 
ax.set_ylim(1)
for p in ax.patches:
    ax.text(p.get_x() + p.get_width()/2., p.get_height(), p.get_height(), 
        fontsize=10, color='black', ha='center', va='bottom')   
plt.show();

In [338]:
new_f = fighters

In [339]:
new_f.rename(columns={'weightCategory.name': 'weight_category_name'}, inplace=True)

In [340]:
women = new_f.weight_category_name.str.contains('Жен')

In [341]:
women1 = len(new_f[women])
men = (len(new_f['weight_category_name'])) - len(new_f[women])

In [342]:
# Соотношение мужчин и женщин среди борцов
labels = 'Men', 'Women'
sizes = [men,women1]
explode = (0, 0.1,)  

fig1, ax1 = plt.subplots(figsize=(10,8))
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90 )
ax1.axis('equal') 

plt.show()

In [343]:
fighters['id'].count()

In [344]:
# Количество бойцов по странам
(fighters
 .pivot_table(index='country', values='id', aggfunc='count')
 .sort_values('id', ascending=False)).head(50)

In [345]:
 fighters['methods.winMethods'].unique()

In [346]:
 fighters['looses'].unique()

In [347]:
 fighters['wins'].unique()

In [348]:
# Посмотрим из каких стран больше всего победителей
(fighters
 .pivot_table(index='country', values='wins', aggfunc='sum')
 .sort_values('wins', ascending=False)).head(50)

# Исследование данных о боях

In [349]:
events.head()

In [350]:
events.columns

In [351]:
events.info()

Пропуски в city, country, duration, rounds, timezone, winnerId

In [352]:
events.describe()

In [353]:
# Посмотрим на информацию в ячейках столбца events['fighters']

In [354]:
events['fighters'].head(2)

In [355]:
events.loc[0, 'fighters']

In [356]:
# Посмотрим на информацию в ячейках столбца events['avgOdds']

In [357]:
events['avgOdds'].unique()

In [358]:
events[events['avgOdds'] == '[]']['avgOdds'].count()

In [359]:
events[events['avgOdds'] != '[]']['avgOdds'].count()

In [360]:
events[events['avgOdds'] != '[]']['avgOdds'].head()

In [361]:
events.loc[1558, 'avgOdds']

# Извлекаем данные из колонок avgOdds и fighters

**Парсим колонку avgOdds**

In [362]:
def parse_odds(row: pd.Series) -> pd.Series:
    """
    Parse 'avgOdds' column.
    :param row: Row of the events dataframe.
    :return: pd.Series with odds for the 1st and the 2nd fighters.
    """
    avg_odds = row["avgOdds"]
    if avg_odds == "[]" or avg_odds == np.nan:
        return pd.Series([np.nan] * 2)
    avg_odds = ast.literal_eval(avg_odds)
    if avg_odds[0]["fighterId"] == row["fighterId_1"]:
        return pd.Series([f.get("value", np.nan) for f in avg_odds])
    else:
        return pd.Series([f.get("value", np.nan) for f in reversed(avg_odds)])

In [363]:
events[["f1_odds", "f2_odds"]] = events[
    ["avgOdds", "fighterId_1", "fighterId_2"]
].apply(lambda row: parse_odds(row), axis=1)
events.drop(columns="avgOdds", inplace=True)
events.head(5)

**Парсим колонку fighters**

In [364]:
fighter_stats_keys = [
    "hitsTotal",
    "hitsSuccessful",
    "takedownTotal",
    "takedownSuccessful",
    "submissionAttempts",
    "takeovers",
    "accentedHitsTotal",
    "accentedHitsSuccessful",
    "knockdowns",
    "protectionPassage",
    "hitsHeadTotal",
    "hitsHeadSuccessful",
    "hitsBodyTotal",
    "hitsBodySuccessful",
    "hitsLegsTotal",
    "hitsLegsSuccessful",
    "accentedHitsPositionDistanceTotal",
    "accentedHitsPositionDistanceSuccessful",
    "accentedHitsPositionClinchTotal",
    "accentedHitsPositionClinchSuccessful",
    "accentedHitsPositionParterTotal",
    "accentedHitsPositionParterSuccessful",
]


def get_fighter_stats_cols() -> List[str]:
    """
    Get list of fight stats column names for each fighter.
    :return: List of column names with 'f1_' prefix
    for the first fighter and 'f2_' prefix for the second.
    """
    fighter_stats_cols = []
    for i in range(1, 3):
        for k in fighter_stats_keys:
            fighter_stats_cols.append(f"f{i}_{k}")
    return fighter_stats_cols


def sum_round_stats(stats: List[Dict[str, int]]) -> List[int]:
    """
    Sum stats for a fighter for all rounds of a fight.
    :param stats: List with stats from object of 'fighters' column.
    :return: Stats for all rounds for a fighter as a list.
    """
    if len(stats) == 0:
        return [np.nan for _ in range(len(fighter_stats_keys))]
    res = {k: 0 for k in fighter_stats_keys}
    for i in stats:
        for k in res:
            res[k] = i.get(k, 0)
    return list(res.values())


def parse_fight_data(row: pd.Series) -> pd.Series:
    """
    Parse 'fighters' column.
    :param row: Row of the events dataframe.
    :return: pd.Series with stats for both fighters.
    """
    fighters = row["fighters"]
    if fighters == "[]" or fighters == np.nan:
        return pd.Series([np.nan for _ in range(len(fighter_stats_keys))])
    cols = []
    fighters = ast.literal_eval(fighters)
    if fighters[0]["fighterId"] == row["fighterId_2"]:
        fighters = reversed(fighters)
    for f in fighters:
        cols.extend(sum_round_stats(f["roundStats"]))
    return pd.Series(cols)

In [365]:
events[get_fighter_stats_cols()] = events[
    ["fighters", "fighterId_1", "fighterId_2"]
].apply(lambda row: parse_fight_data(row), axis=1)
events.drop(columns="fighters", inplace=True)
events.head(5)

**Добавляем данные о бойцах в датафрейм с боями**

In [366]:
fighter_data_cols = fighters.drop(
    columns=["weightCategory.id"]
).columns
events = events.join(
    fighters[fighter_data_cols].add_prefix("f1_"), on="fighterId_1"
)
events = events.join(
    fighters[fighter_data_cols].add_prefix("f2_"), on="fighterId_2"
)
events.head(5)

**Добавляем признак age**

In [367]:
def add_age(row: pd.Series) -> pd.Series:
    """
    Add age for both fighters.
    :param row: Row of the events dataframe.
    :return: pd.Series with age of fighters in years.
    """
    result = []
    for prefix in ["f1_", "f2_"]:
        try:
            age = row["eventDate.date"].year - row[prefix + "dateOfBirth"].year
        except Exception:
            age = np.nan
        result.append(age)
    return pd.Series(result)

In [368]:
events['eventDate.date'] = pd.to_datetime(events['eventDate.date'])

In [369]:
events[["f1_age", "f2_age"]] = events[
    ["eventDate.date", "f1_dateOfBirth", "f2_dateOfBirth"]
].apply(lambda row: add_age(row), axis=1)

In [370]:
events = events[(events['f1_age'] >= 18) & (events['f2_age'] >= 18)]
events.head(5)

In [371]:
final_df = events[[ 
    'duration', 
    'eventDate.date', 
    'fighterId_1',
    'fighterId_2',
    'id',
    'rounds',
    'weightCategory.id',
    'winnerId',   
    "f1_hitsTotal",
    "f1_hitsSuccessful",
    "f1_takedownTotal",
    "f1_takedownSuccessful",
    "f1_submissionAttempts",
    "f1_takeovers",
    "f1_accentedHitsTotal",
    "f1_accentedHitsSuccessful",
    "f1_knockdowns",
    "f2_knockdowns",
    "f1_weight",
    "f2_weight",
    "f1_age",
    "f2_age"
]]

In [372]:
# When fighterID_1 wins, then Winner == 1 
final_df['winner'] = final_df['winnerId'] == final_df['fighterId_1']

In [373]:
final_df.head()

In [374]:
final_df["year"] = final_df["eventDate.date"].dt.year
df_test = final_df[final_df['year'] == 2020]
df_train = final_df[final_df['year'] != 2020]
df_test.head()

In [375]:
X_train = df_train.drop(columns="winner")
y_train = df_train["winner"]

X_test = df_test.drop(columns="winner")
y_test = df_test["winner"]


In [376]:
clf = CatBoostClassifier(iterations=1000, 
                           depth=4,
                           learning_rate=0.01,
                           loss_function='Logloss',
                           colsample_bylevel=1,
                           subsample=0.5,
#                                    bagging_temperature=bagging_temperature,
                           random_strength=1,
                           l2_leaf_reg=30,
                           eval_metric='Accuracy',
#                            boosting_type='Plain', # 'Plain'
                           bootstrap_type='Bernoulli',# 'Bernoulli' Bayesian
                           max_ctr_complexity=2,
                           task_type='GPU',
                          )

clf.fit(X_train, y_train, plot=True, eval_set=(X_test, y_test), verbose=False)

In [377]:
y_pred = clf.predict(df_test)
df_test['y_pred'] = y_pred
df_test

In [378]:
print('Точность модели')
(df_test['y_pred'] == df_test['winner']).mean()

In [379]:
len(df_test.loc[ df_test['y_pred']==df_test['winner']])

In [380]:
df_test.loc[df_test['y_pred'] == True, 'y_pred'] = 1
df_test.loc[df_test['y_pred'] == False, 'y_pred'] = 0
df_test.loc[df_test['winner'] == True, 'winner'] = 1
df_test.loc[df_test['winner'] == False, 'winner'] = 0

In [381]:
print('Точность модели')
(df_test['y_pred'] == df_test['winner']).mean()

In [382]:
len(df_test.loc[ df_test['y_pred']==df_test['winner']])